In [2]:
import pandas as pd
import numpy as np
import faiss
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
import math



In [3]:
# loading the entire test embeddings
path = "../5_embeddings/cls_embeddings_time.struct_time(tm_year=2025, tm_mon=2, tm_mday=7, tm_hour=19, tm_min=40, tm_sec=22, tm_wday=4, tm_yday=38, tm_isdst=0).npy"

cls_embeddings = np.load(path)
print(cls_embeddings.shape)


(118108, 32)


In [4]:
train_df = pd.read_csv("../2_dataset/final/train_df.csv")
train_df

,cls,ProductCD,card4,card6,P_emaildomain,Unnamed: 0,TransactionDT,TransactionAmt,card1,card2,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,0,4,2,1,2,-0.291883,-0.329939,0.108390,-0.145421,-0.399322,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
1,0,4,3,2,16,0.892993,0.871243,-0.359702,0.680504,-0.412094,...,-0.030054,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
2,0,4,2,2,1,-1.594876,-1.467121,8.134522,-0.109308,0.711822,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
3,0,4,3,2,19,-0.123148,-0.156138,-0.422421,1.487250,-0.265218,...,0.341765,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
4,0,4,3,2,16,1.611964,1.677853,-0.317889,-0.081355,-0.265218,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76765,0,4,3,2,16,-1.327921,-1.260784,-0.113217,-0.653259,-1.606254,...,1.009878,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
76766,0,4,2,2,25,0.675641,0.648266,-0.075376,-0.002802,0.756523,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
76767,0,4,1,1,16,0.418154,0.377752,0.150412,-1.485918,-0.226903,...,-0.227583,0.393449,0.090945,0.276274,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142
76768,0,4,3,2,19,0.605578,0.576883,-0.322279,-0.182963,0.577719,...,-0.227583,-0.222385,-0.249222,-0.229148,-0.048377,-0.062211,-0.058049,-0.055287,-0.088855,-0.074142


In [5]:
# train_y = pd.read_csv("./data/train_y_df.csv")
# train_y['isFraud'] = train_y['isFraud'].astype(np.float32)
# train_y.shape
train_y = pd.read_csv("../2_dataset/final/train_y_df.csv")
# train_y['isFraud'] = train_y['isFraud'].astype(np.float32)
train_y.shape


(76770, 1)

In [6]:
test_df = pd.read_csv("../2_dataset/final/test_df.csv")
test_df.shape

(23622, 182)

In [7]:
test_y_df = pd.read_csv("../2_dataset/final/test_y_df.csv")
test_y_df['isFraud'] = test_y_df['isFraud'].astype(np.float32)


In [8]:
test_y_df['isFraud']


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
23617    0.0
23618    0.0
23619    1.0
23620    0.0
23621    0.0
Name: isFraud, Length: 23622, dtype: float32

In [9]:
val_df = pd.read_csv("../2_dataset/final/val_df.csv")
val_y_df = pd.read_csv("../2_dataset/final/val_y_df.csv")

#### Data exploration

In [10]:
train_combined = pd.concat([train_df, train_y], axis=1)
for i in train_combined.columns:
    print(i)


cls
ProductCD
card4
card6
P_emaildomain
Unnamed: 0
TransactionDT
TransactionAmt
card1
card2
card3
card5
addr1
addr2
C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
C13
C14
D1
D10
D15
V12
V13
V14
V15
V16
V17
V18
V19
V20
V21
V22
V23
V24
V25
V26
V27
V28
V29
V30
V31
V32
V33
V34
V53
V54
V55
V56
V57
V58
V59
V60
V61
V62
V63
V64
V65
V66
V67
V68
V69
V70
V71
V72
V73
V74
V75
V76
V77
V78
V79
V80
V81
V82
V83
V84
V85
V86
V87
V88
V89
V90
V91
V92
V93
V94
V95
V96
V97
V98
V99
V100
V101
V102
V103
V104
V105
V106
V107
V108
V109
V110
V111
V112
V113
V114
V115
V116
V117
V118
V119
V120
V121
V122
V123
V124
V125
V126
V127
V128
V129
V130
V131
V132
V133
V134
V135
V136
V137
V279
V280
V281
V282
V283
V284
V285
V286
V287
V288
V289
V290
V291
V292
V293
V294
V295
V296
V297
V298
V299
V300
V301
V302
V303
V304
V305
V306
V307
V308
V309
V310
V311
V312
V313
V314
V315
V316
V317
V318
V319
V320
V321
isFraud


#### splitting embeddings

In [11]:
# Compute 65% of the total rows
total_rows = cls_embeddings.shape[0]
train_size = int(0.65 * total_rows)  # 65% of 118108

# Slice the top 65%
train_embeddings = cls_embeddings[:train_size]  # First 65%

print(f"Total embed shape: {cls_embeddings.shape}")
print(f"Train embed shape: {train_embeddings.shape}")


Total embed shape: (118108, 32)
Train embed shape: (76770, 32)


In [12]:
total_rows = cls_embeddings.shape[0]
test_size = int(0.8 * total_rows) 

test_embeddings = cls_embeddings[test_size:]
print(f"Total embed shape: {cls_embeddings.shape}")
print(f"Train embed shape: {test_embeddings.shape}")


Total embed shape: (118108, 32)
Train embed shape: (23622, 32)


In [13]:
val_embeddings = cls_embeddings[train_size:test_size]
len(val_embeddings) # 15% of total rows.

17716

#### multiple indexes

In [14]:
len(train_embeddings), int(math.sqrt(len(train_embeddings)))  # FAISS default approach


(76770, 277)

In [15]:
# nlist = 50  # how many cells
n_clusters = int(math.sqrt(len(train_embeddings)))  # FAISS default approach
dimension = 32
quantizer = faiss.IndexFlatL2(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, n_clusters)


In [16]:
index.is_trained


False

In [17]:
index.train(train_embeddings)
# index.is_trained  # check if index is now trained


#### faiss index and similarity search

In [18]:
def create_index(num_embeddings, dimension):


    # num_embeddings = 76770
    # dimension = 32

    index = faiss.IndexFlatL2(dimension)  # L2 similarity

    index.add(train_embeddings)  # index of pre-computed embeddings

    # k = 120  # as best result for 120

    return index


# query_vector = np.random.random((1, dimension)).astype("float32") #random for now


# print(k)


In [19]:
def search_faiss(query_vector):

    index = create_index(76770, 32)

    # Convert PyTorch tensor to NumPy
    if isinstance(query_vector, torch.Tensor):
        query_vector = query_vector.detach().cpu().numpy()

    query_vector = query_vector.astype("float32").reshape(1, -1)


    query_vector = query_vector.astype("float32").reshape(1, -1)

    distances, indices = index.search(query_vector, k=120) # by default using euclidean distance for similarity
    indices = indices.flatten()


    return distances, indices

# distances, indices = index.search(
#     query_vector, k
# )  

# print("Input Sample embedding:", query_vector)
# print("Indices of nearest neighbors:", indices)
# print("L2 norm distances", distances)


#### L2 distance component

In [20]:
def compute_similarity(distances, dropout=0.2):

    # print(f"distances.shape before: {distances.shape}")
    distances = distances.flatten()
    # print(f"distances.shape after: {distances.shape}")

    # Apply softmax to the negative distances
    similarities = np.exp(-distances)
    softmax_scores = similarities / np.sum(similarities)

    # Apply dropout (randomly zero out some softmax scores)
    dropout_mask = np.random.binomial(1, 1 - dropout, size=softmax_scores.shape)
    dropped_softmax_scores = softmax_scores * dropout_mask

    final_softmax = dropped_softmax_scores / np.sum(dropped_softmax_scores)
    # how to weigh in the final embedding?
    return final_softmax, distances


#### mask to drop the dropped out values

In [21]:
def filter_by_mask(arr1, arr2, arr3):
    """
    Removes elements from arr2 and arr3 where corresponding indices in arr1 are zero.
    """
    mask = arr1 != 0  # Create a boolean mask where arr1 is nonzero
    return arr1[mask], arr2[mask], arr3[mask]

# S_x_xi, indices, distances = filter_by_mask(S_x_xi, indices, distances)

# print("Filtered S_x_xi:", S_x_xi)
# print("Filtered indices:", indices)
# print("Filtered distances:", distances)



#### linear trasnform the value component

In [22]:
class MLP_L1(nn.Module):
    def __init__(self, input_dim):
        super(MLP_L1, self).__init__()
        self.layer1 = nn.Linear(input_dim, 32)
        self.activation1 = nn.SiLU()

        self.layer2 = nn.Linear(32, 32)
        self.activation2 = nn.SiLU()
        self.dropout2 = nn.Dropout(p=0.2)

        self.layer3 = nn.Linear(32, 32)  # Fix: Output should be 32
        self.activation3 = nn.SiLU()  # Fix: Apply SiLU activation

    def forward(self, x):
        x = self.activation1(self.layer1(x))
        x = self.dropout2(self.activation2(self.layer2(x)))
        x = self.activation3(self.layer3(x))  # Fix: Apply activation & dropout
        return x


# # Create model instance
# model = MLP_L1(l1_dist.shape[0])

# # Convert input to tensor and pass it through the model
# l1_dist_tensor = torch.tensor(l1_dist, dtype=torch.float32)

# w_v_l1 = model(l1_dist_tensor).detach().numpy()
# print(w_v_l1)

# print(w_v_l1.shape)


In [23]:
def compute_l1(distances):
    l1_dist = np.sqrt(distances)
    # print(l1_dist, l1_dist.shape)

    model = MLP_L1(l1_dist.shape[0])

    # Convert input to tensor and pass it through the model
    l1_dist_tensor = torch.tensor(l1_dist, dtype=torch.float32)

    w_v_l1 = model(l1_dist_tensor).detach().numpy()
    # print(w_v_l1)

    # print(w_v_l1.shape)

    return w_v_l1


In [24]:
# Modified MLP to output shape (60,)
class MLP_Wy(nn.Module):
    def __init__(self, input_dim):
        super(MLP_Wy, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # First layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 32)  # Output layer (1 neuron)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)  # Shape (60, 1)
        return x.squeeze(1)  # Shape (60,)

# Instantiate MLP with input_dim=32 (from Wy)
 # Expected: (60,)


In [25]:
train_y


,isFraud
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
76765,0.0
76766,0.0
76767,0.0
76768,1.0


In [26]:
def compute_wy(indices):
    y_i = train_y['isFraud'].loc[indices].values
    # print(f"y_i.shape: {y_i.shape}")


# Convert to PyTorch tensor
    feature_target_tensor = torch.tensor(y_i, dtype=torch.long)

# Define Wy: An embedding layer to map to 32-dim space
    embedding_dim = 32
    num_classes = 2  # Since input values are 0 or 1

    Wy = nn.Embedding(num_classes, embedding_dim)

    mlp = MLP_Wy(input_dim=embedding_dim)

# Compute embeddings using Wy
    embeddings = Wy(feature_target_tensor)  # Shape: (60, 32)

    # Pass embeddings through MLP
    w_y = mlp(embeddings)  # Shape: (60,)

    # print("MLP Output Shape:", w_y.shape) 

    return w_y


In [27]:
def compute_value(w_v_l1, w_y):
    # Compute the dot product of w_v_l1 and w_y
    w_y_npy = w_y.detach().numpy()
    value = w_y_npy + w_v_l1
    return value


#### Reshaping S to do S * V

In [28]:
def compute_z_in(S_x_xi, value):
    S_x_xi = S_x_xi.reshape(1, -1)
    # print(S_x_xi.shape)  # Output should be (1, 95)

    # result = (S_x_xi @ value)

    # # Summation over all elements (since it's 1D)
    # Z = np.sum(result)

    # Assuming S_x_xi is (1, 95) and value is (95, 32)
    numerator = np.sum(S_x_xi @ value)  # Sum the weighted contributions (scalar)
    denominator = np.sum(S_x_xi)        # Total sum of weights (scalar)
    z_in = numerator / denominator         # Weighted average as a single scalar

    # print("Z:", z_in)
    # print("Z Shape:", z_in.shape)  # Expected: torch.Size([])


    return z_in


In [29]:
# x_test = test_df.iloc[44].values
# x_test.shape


### Model Training

In [30]:
import torch
import torch.nn as nn

class ThreeBlockModel(nn.Module):
    def __init__(self, input_emb_dim, hidden_dim=32, dropout_prob=0.2):
        super(ThreeBlockModel, self).__init__()
        
        self.input_dim = input_emb_dim + 1  # Adding 2 for weighted_avg and f_z_in

        self.block1 = nn.Sequential(
            nn.LayerNorm(self.input_dim),
            nn.Linear(self.input_dim, hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
        )
        
        self.block2 = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
        )
    
    
        
        self.block3 = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.SiLU(),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, hidden_dim),
        )
        
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    
    def forward(self, input_emb, weighted_avg):
        # Ensure correct shape for scalar inputs
        weighted_avg = weighted_avg.unsqueeze(-1)  
    
        # Concatenate all inputs
        combined = torch.cat([input_emb, weighted_avg], dim=-1)

        # Pass through MLP blocks
        x = self.block1(combined)
        x = self.block2(x)
        x = self.block3(x)
        x = self.output_layer(x)
        return x


In [31]:
# def process_samples(test_df, test_embeddings):
#     test_i = test_df.iloc[i].values #shape (182,)
#     query_vector = test_embeddings[i] #shape (32,)
#     distances, indices = search_faiss(query_vector) # both shape (120,) and flatten
#     S_x_xi, distances = compute_similarity(distances)
#     S_x_xi, indices, distances = filter_by_mask(S_x_xi, indices, distances)
#     w_v_l1 = compute_l1(distances) #shape (32,)
#     w_y = compute_wy(indices) #shape (32,)
#     value = compute_value(w_v_l1, w_y)
#     z_in = compute_z_in(S_x_xi, value)

#     input_to_mlp = test_i * z_in


In [5]:
test_i = test_df.iloc[10].values #shape (182,)
query_vector = test_embeddings[10] #shape (32,)
distances, indices = search_faiss(query_vector) # both shape (120,) and flatten
S_x_xi, distances = compute_similarity(distances)
S_x_xi, indices, distances = filter_by_mask(S_x_xi, indices, distances)
w_v_l1 = compute_l1(distances) #shape (32,)
w_y = compute_wy(indices) #shape (32,)


feat_x_1, feat_x2, feat_x3, feat_x4 = compute_imp_features(indices=indices)
#z_in 
f_z_1 = compute_z_in(S_x_xi, feat_x_1)
f_z_2 = compute_z_in(S_x_xi, feat_x2)
f_z_3 = compute_z_in(S_x_xi, feat_x3)
f_z_4 = compute_z_in(S_x_xi, feat_x4)



NameError: name 'test_df' is not defined

In [6]:
f_z_1, f_z_2, f_z_3, f_z_4


NameError: name 'f_z_1' is not defined

In [7]:
value = compute_value(w_v_l1, w_y)
value.shape


NameError: name 'compute_value' is not defined

In [8]:
labels = test_y_df['isFraud']
labels = torch.tensor(test_y_df['isFraud'].values, dtype=torch.float32)
type(labels)

type(test_df.iloc[34].values)


NameError: name 'test_y_df' is not defined

In [32]:
def process_samples(test_df, test_embeddings):
    """Processes all samples and returns input tensor and labels."""
    input_list = []
    labels = []


    for i in range(len(test_df)):  # Process all samples
        test_i = torch.tensor(test_df.iloc[i].values, dtype=torch.float32)  # shape (182,)
        query_embedding = torch.tensor(test_embeddings[i], dtype=torch.float32)  # shape (32,)

        distances, indices = search_faiss(query_embedding)
        S_x_xi, distances = compute_similarity(distances)
        S_x_xi, indices, distances = filter_by_mask(S_x_xi, indices, distances)
        w_v_l1 = compute_l1(distances)  # shape (32,)
        w_y = compute_wy(indices)  # shape (32,)
        value = compute_value(w_v_l1, w_y)
        z_in = compute_z_in(S_x_xi, value)  # Scalar (single value)


        # Append only test_i (input_emb) and z_in (weighted_avg), not query_embedding
        z_in = torch.tensor(z_in, dtype=torch.float32)

        input_list.append((test_i, z_in))

        labels.append(torch.tensor(train_y.iloc[i]['isFraud'], dtype=torch.float32))  # Assuming label is in df



    return input_list, labels


In [33]:
def val_process_samples(test_df, test_embeddings):
    """Processes all samples and returns input tensor and labels."""
    input_list = []
    labels = []


    for i in range(len(test_df)):  # Process all samples
        test_i = torch.tensor(test_df.iloc[i].values, dtype=torch.float32)  # shape (182,)
        query_embedding = torch.tensor(test_embeddings[i], dtype=torch.float32)  # shape (32,)

        distances, indices = search_faiss(query_embedding)
        S_x_xi, distances = compute_similarity(distances)
        S_x_xi, indices, distances = filter_by_mask(S_x_xi, indices, distances)
        w_v_l1 = compute_l1(distances)  # shape (32,)
        w_y = compute_wy(indices)  # shape (32,)
        value = compute_value(w_v_l1, w_y)
        z_in = compute_z_in(S_x_xi, value)  # Scalar (single value)


        # Append only test_i (input_emb) and z_in (weighted_avg), not query_embedding
        z_in = torch.tensor(z_in, dtype=torch.float32)

        input_list.append((test_i, z_in))

        labels.append(torch.tensor(val_y_df.iloc[i]['isFraud'], dtype=torch.float32))  # Assuming label is in df



    return input_list, labels


In [11]:
# import torch.optim as optim
# import torch.nn as nn
# from torch.utils.data import DataLoader, TensorDataset

# def train_model(test_df, test_embeddings, model, optimizer, criterion, batch_size=256, epochs=25):
#     model.train()
    
#     # Process data
#     # input_list, labels = process_samples(test_df, test_embeddings)

#     # Create DataLoader
#     dataset = TensorDataset(
#         torch.stack([item[0] for item in input_list]),  # test_i (input_emb)
#         torch.stack([item[1] for item in input_list]),  # z_in (weighted_avg)
#         torch.stack(labels)  # Labels
#     )
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#     # Training loop for multiple epochs
#     for epoch in range(epochs):
#         epoch_loss = 0  # Track loss for each epoch

#         for batch in dataloader:
#             input_emb, z_in, target = batch
#             target = target.unsqueeze(-1)  # Make target shape (batch_size, 1)

#             optimizer.zero_grad()
#             output = model(input_emb, z_in)

#             loss = criterion(output, target)
#             loss.backward()
#             optimizer.step()

#             epoch_loss += loss.item()  # Accumulate loss for epoch

#         print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(dataloader):.4f}")  # Print average loss per epoch

# # Model, optimizer, and loss function
# input_emb_dim = 182  # Assuming this based on test_df features
# model = ThreeBlockModel(input_emb_dim=input_emb_dim)
# optimizer = optim.Adam(model.parameters(), lr=0.0001)
# criterion = nn.BCELoss()  # Binary Cross Entropy Loss

# # Call training loop (assuming test_df and test_embeddings are available)
# train_model(test_df, test_embeddings, model, optimizer, criterion, epochs=100)


In [ ]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import average_precision_score
import torch

def train_model(test_df, test_embeddings, model, optimizer, criterion, batch_size=256, epochs=25):
    model.train()
    
    # Process data
    # TODO!!!!!
    input_list, labels = process_samples(train_df, train_embeddings)
    val_input_list, val_labels = val_process_samples(val_df, val_embeddings)


    # Create DataLoader
    dataset = TensorDataset(
        torch.stack([item[0] for item in input_list]),  # test_i (input_emb)
        torch.stack([item[1] for item in input_list]),  # z_in (weighted_avg)
        torch.stack(labels)  # Labels
    )
    val_dataset = TensorDataset(
        torch.stack([item[0] for item in val_input_list]),  # test_i (input_emb)
        torch.stack([item[1] for item in val_input_list]),  # z_in (weighted_avg)
        torch.stack(val_labels)  # Labels
    )

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Training loop for multiple epochs
    for epoch in range(epochs):
        epoch_loss = 0  # Track loss for each epoch
        
        all_targets = []
        all_outputs = []

        for batch in dataloader:
            input_emb, z_in, target = batch
            target = target.unsqueeze(-1)  # Make target shape (batch_size, 1)

            optimizer.zero_grad()
            output = model(input_emb, z_in)

            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()  # Accumulate loss for epoch
            
            # Collect predictions & targets for AUCPR
            all_outputs.append(output.detach().cpu())  # Move to CPU to avoid memory issues
            all_targets.append(target.detach().cpu())

        # Compute AUCPR at the end of the epoch
        all_outputs = torch.cat(all_outputs).numpy()
        all_targets = torch.cat(all_targets).numpy()

        aucpr = average_precision_score(all_targets, all_outputs)

         # Validation Step
        model.eval()
        val_loss = 0
        all_val_targets = []
        all_val_outputs = []

        with torch.no_grad():  # No gradient computation for validation
            for batch in val_dataloader:
                input_emb, z_in, f_z_in, f_z_in_2, f_z_in_3, f_z_in_4, target = batch
                target = target.unsqueeze(-1)

                output = model(input_emb, z_in, f_z_in, f_z_in_2, f_z_in_3, f_z_in_4)
                loss = criterion(output, target)

                val_loss += loss.item()
                
                all_val_outputs.append(output.cpu())
                all_val_targets.append(target.cpu())

        # Compute AUCPR for validation set
        all_val_outputs = torch.cat(all_val_outputs).numpy()
        all_val_targets = torch.cat(all_val_targets).numpy()
        val_aucpr = average_precision_score(all_val_targets, all_val_outputs)

        print(f"Epoch [{epoch+1}/{epochs}], "
              f"Train Loss: {epoch_loss/len(dataloader):.4f}, Train AUCPR: {aucpr:.4f}, "
              f"Val Loss: {val_loss/len(val_dataloader):.4f}, Val AUCPR: {val_aucpr:.4f}")
        
# Model, optimizer, and loss function
input_emb_dim = 182  # Assuming this based on test_df features
model = ThreeBlockModel(input_emb_dim=input_emb_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss

# Call training loop (assuming test_df and test_embeddings are available)
train_model(train_df, train_embeddings, model, optimizer, criterion, epochs=100)
